In [1]:
import re
import matplotlib.image as mpimg
import numpy as np
from PIL import Image
import cil_helper as cil
import os

In [2]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.75 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = np.sum(v)
    if df < foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

# Extract label images
def extract_labels(filename, num_images, IMG_PATCH_SIZE):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [cil.img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

data_dir = 'training/'
train_labels_filename = data_dir + 'groundtruth/' 

In [3]:
PIXEL_DEPTH = 255
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg

In [4]:
def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

In [5]:
# assign a label to a patch
def patch_to_label(patch):
    th = 0.25
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0

In [6]:
def summary_to_label(patch, th):
    df = np.mean(patch)
    if df >= th:
        return 1
    else:
        return 0

In [7]:
def mask_to_submission_strings(im, img_number):
    """Reads a single image and outputs the strings that should go into the submission file"""
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number+1, j, i, label))

In [8]:
def masks_to_submission(submission_filename, *image_files):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')         
        for ind, fn in enumerate(image_files[0:]):
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn, ind))

In [9]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])
def collect_masks(name, ending, patches, id, th):
    train_labels = extract_labels(train_labels_filename, TRAINING_SIZE+VALIDATION_SIZE, 16)
    TEST_IMG_SIZE = 400
    img = np.zeros((TEST_IMG_SIZE,TEST_IMG_SIZE))
    for p in patches:
        imagename = name + str(p) + '_' + str(id) + ending
        #print(imagename)
        image = mpimg.imread(imagename)
        image = image[:,TEST_IMG_SIZE:,:]
        image = rgb2gray(image) 
        for j in range(0, image.shape[1], p):
            for i in range(0, image.shape[0], p):
                label = np.mean(image[i:i + p, j:j + p])
                #pred_label = np.mean(img[i:i + p, j:j + p])
                # set new label
                img[i:i + p, j:j + p] = img[i:i + p, j:j + p]+label
    
    p = 16
    labels = []
    for j in range(0, img.shape[1], p):
            for i in range(0, img.shape[0], p):
                patch = img[i:i + p, j:j + p]
                label = summary_to_label(patch,th)
                img[i:i + p, j:j + p] = label
                labels.append([label, 1-label])
                
    # error calc 
    labels = np.asarray(labels)
    err = cil.error_rate(labels, train_labels[labels.shape[0]*id:labels.shape[0]*(id+1)])
    print(err)   
    error.append(err)
            
    # make overlay
    testname = 'training/images/satImage_0' + str(id) + '.png'
    testimg = mpimg.imread(testname)        
    oimg = make_img_overlay(testimg, img)
    oimg.save('predictions_training/' + "overlay-post-comb_" + str(id) + ".png")
                
    return img

In [10]:
TRAINING_SIZE = 40
VALIDATION_SIZE = 10  # Size of the validation set.
def save(submission_filename, th):
    #subm#ission_filename = 'submission_07.csv'
    patches = [2,8,16]
    images = []
    for i in range(TRAINING_SIZE, TRAINING_SIZE+VALIDATION_SIZE):
    #for i in range(40, 41):
        images.append(collect_masks('predictions_training/prediction__eval_postproc', '.png', patches, i, th))
    masks_to_submission(submission_filename, *images)


In [18]:
error = []
save('submission_eval_lukas.csv', 1)
print("Mean: " + str(np.mean(error)))

44.32
50.88
44.0
42.88
40.96
37.28
40.48
46.4
42.88
43.04
Mean: 43.312


In [19]:
error = []
save('submission_eval_lukas.csv', 1.5)
print("Mean: " + str(np.mean(error)))

32.16
34.88
40.8
31.84
30.24
24.48
27.84
36.96
32.96
30.56
Mean: 32.272


In [20]:
error = []
save('submission_eval_lukas.csv', 2)
print("Mean: " + str(np.mean(error)))

23.68
29.76
34.72
26.88
20.8
16.64
22.4
31.2
28.0
25.76
Mean: 25.984


In [21]:
error = []
save('submission_eval_lukas.csv', 2.5)
print("Mean: " + str(np.mean(error)))

17.28
24.8
27.84
20.8
13.92
12.32
18.88
25.92
23.2
19.84
Mean: 20.48


In [22]:
error = []
save('submission_eval_lukas.csv', 2.75)
print("Mean: " + str(np.mean(error)))

12.64
23.36
23.84
16.96
9.76
11.52
18.24
22.56
20.32
16.32
Mean: 17.552


In [23]:
error = []
save('submission_eval_lukas.csv', 2.95)
print("Mean: " + str(np.mean(error)))

9.6
21.44
21.76
14.24
7.2
10.24
16.96
20.48
16.8
11.84
Mean: 15.056


In [24]:
error = []
save('submission_eval_lukas.csv', 2.7)
print("Mean: " + str(np.mean(error)))

13.28
23.68
24.64
18.08
10.24
12.0
17.92
23.84
20.48
17.44
Mean: 18.16
